### 测试求手物交互的中间表示，以及反向通过中间表示，优化获得最终的手物参数

In [2]:
import torch
from torch.utils import data
import numpy as np
import os
from os.path import join 
import random
from tqdm import *
import pickle
import trimesh
from tqdm import *
from manotorch.manolayer import ManoLayer
from pytorch3d.transforms import axis_angle_to_matrix
from random import choice
from sklearn.neighbors import KDTree
from pysdf import SDF
manolayer = ManoLayer(
    mano_assets_root='/root/code/CAMS/data/mano_assets/mano',
    side='right'
)

def convert_T_to_obj_frame(points, obj_pose):
    # points(frames,21,3)
    # obj_pose (frames,3,4)

    obj_T = obj_pose[:,:3,3].unsqueeze(-2) # B, 1, 3
    points = points - obj_T
    points = torch.einsum('...ij->...ji', [points])
    obj_R = obj_pose[:,:3,:3] # B, 3, 3
    obj_R = torch.einsum('...ij->...ji', [obj_R])
    new_points = torch.einsum('bpn,bnk->bpk',obj_R,points)
    new_points = torch.einsum('...ij->...ji', [new_points])
    return new_points

def convert_R_to_obj_frame(hand_rot, obj_pose):
    # hand_rot: B，3，3
    obj_R = obj_pose[:,:3,:3] # B, 3, 3
    obj_R = torch.einsum('...ij->...ji', [obj_R])
    hand_rot_in_obj = torch.einsum('bji,bjk->bik', obj_R, hand_rot)

    return hand_rot_in_obj

# def compute_angular_velocity_for(rotation_matrices):
#     # rotation_matrices: (T, 3, 3), where T is the number of time steps
#     T = rotation_matrices.shape[0]
    
#     # Initialize angular velocity array
#     angular_velocity = torch.zeros((T - 1, 3, 3))
    
#     for i in range(T - 1):
#         R_next = rotation_matrices[i + 1]
#         R_current = rotation_matrices[i]
#         R_diff = R_next @ R_current.transpose(-1, -2) - torch.eye(3)
#         angular_velocity[i] = R_diff 
        
#     return angular_velocity

def compute_angular_velocity_nofor(rotation_matrices):
    # rotation_matrices: (T, 3, 3), where T is the number of time steps
    R_next = rotation_matrices[1:]  # (T-1, 3, 3)
    R_current = rotation_matrices[:-1]  # (T-1, 3, 3)
    
    # Compute difference matrix R_next * R_current^T - I
    R_diff = R_next @ R_current.transpose(-1, -2) - torch.eye(3).to(rotation_matrices.device)
    
    # Extract the angular velocity matrix (anti-symmetric part)
    angular_velocity = R_diff
    
    return angular_velocity

def compute_angular_acceleration_nofor(angular_velocity):
    # angular_velocity: (T-1, 3, 3), where T-1 is the number of time steps
    omega_next = angular_velocity[1:]  # (T-2, 3, 3)
    omega_current = angular_velocity[:-1]  # (T-2, 3, 3)
    
    # Compute the difference in angular velocity
    omega_diff = omega_next - omega_current
    
    # Compute angular acceleration
    angular_acceleration = omega_diff
    
    return angular_acceleration


datapath='/root/code/seqs/gazehoi_list_train_new.txt'
root = '/root/code/seqs/1205_data/'
obj_path = '/root/code/seqs/object/'
with open(datapath,'r') as f:
    info_list = f.readlines()
seqs = []
for info in info_list:
    seq = info.strip()
    seqs.append(seq)
datalist = []
# print(seqs)
for seq in seqs:
    seq_path = join(root,seq)
    meta_path = join(seq_path,'meta.pkl')
    mano_right_path = join(seq_path, 'mano/poses_right.npy')
    with open(meta_path,'rb')as f:
        meta = pickle.load(f)
    
    active_obj = meta['active_obj']
    obj_mesh_path = join(obj_path,active_obj,'simplified_scan_processed.obj')
    obj_mesh = trimesh.load(obj_mesh_path)
    obj_sdf = SDF(obj_mesh.vertices,obj_mesh.faces)
    obj_verts = torch.tensor(np.load(join(obj_path,active_obj,'resampled_500_trans.npy'))).float()
    obj_pose = torch.tensor(np.load(join(seq_path,active_obj+'_pose_trans.npy'))).float()

    hand_params = torch.tensor(np.load(mano_right_path)).float()
    hand_trans = hand_params[:,:3]
    hand_rot = hand_params[:,3:6]
    hand_rot_matrix = axis_angle_to_matrix(hand_rot)
    print(hand_rot_matrix.shape)
    hand_theta = hand_params[:,3:51]
    mano_beta = hand_params[:,51:]
    mano_output = manolayer(hand_theta, mano_beta)
    hand_joints = mano_output.joints - mano_output.joints[:, 0].unsqueeze(1) + hand_trans.unsqueeze(1) # B, 21, 3
    # 物体坐标系下的手部关键点
    hand_joints_in_obj = convert_T_to_obj_frame(hand_joints, # B, 21, 3
                                        obj_pose)           # B, 3, 4
    print(hand_joints_in_obj.shape)
    # 手物是否接触 0-1值
    hand_contact = obj_sdf(hand_joints_in_obj.reshape(-1,3)).reshape(-1,21) < 0.02 # TODO: 阈值调整
    print(hand_contact.shape)

    # 手物之间的offset
    # hand_joints_in_obj.unsqueeze(2) - obj_verts.unsqueeze(1)
    hand_obj_dis = torch.norm(hand_joints_in_obj.unsqueeze(2) - obj_verts.unsqueeze(0).unsqueeze(0).repeat(hand_params.shape[0],1,1,1),dim=-1) # B,21,1,3 - B,1,500,3 = B,21,500
    print(hand_obj_dis.shape)
    obj_ids = torch.argmin(hand_obj_dis,dim=-1) # B,21

    closest_obj_verts = obj_verts[obj_ids] # B,21,3
    print(closest_obj_verts.shape)
    hand_obj_offset = hand_joints_in_obj - closest_obj_verts

    # 手部21个节点的线速度 线加速度
    hand_lin_vel = hand_joints_in_obj[1:] - hand_joints_in_obj[:-1] # B-1,21,3
    hand_lin_acc = hand_lin_vel[1:] - hand_lin_vel[:-1] # B-2,21,3
    print(hand_lin_vel.shape, hand_lin_acc.shape)

    # 手部根节点的角速度 角加速度
    #  需要将手部的旋转也转到物体坐标系下
    hand_rot_in_obj = convert_R_to_obj_frame(hand_rot_matrix, obj_pose)
    # print(f'hand_pose dtype: {hand_rot_matrix.dtype}')
    # print(f'obj_pose dtype: {obj_pose.dtype}')
    # 计算角速度和角加速度 
    # 验证了nofor的结果是正确的，最大误差5.9605e-08
    # ang_vel_for = compute_angular_velocity_for(hand_rot_in_obj)
    # ang_vel_nofor = compute_angular_velocity_nofor(hand_rot_in_obj)
    # print(ang_vel_for.shape,ang_vel_nofor.shape)
    # print(torch.max(ang_vel_for - ang_vel_nofor))
    hand_ang_vel = compute_angular_velocity_nofor(hand_rot_in_obj)
    hand_ang_acc = compute_angular_acceleration_nofor(hand_ang_vel)





    break

torch.Size([139, 3, 3])
torch.Size([139, 21, 3])
(139, 21)
torch.Size([139, 21, 500])
torch.Size([139, 21, 3])
torch.Size([138, 21, 3]) torch.Size([137, 21, 3])


### 统计motion长度


In [3]:
seq_lens = []
for seq in seqs:
    seq_path = join(root,seq)
    meta_path = join(seq_path,'meta.pkl')
    mano_right_path = join(seq_path, 'mano/poses_right.npy')
    hand_params = np.load(mano_right_path)
    seq_len = hand_params.shape[0]
    seq_lens.append(seq_len)

In [4]:
min(seq_lens), max(seq_lens)

(84, 345)

In [10]:
import numpy as np
import matplotlib.pyplot as plt

# 示例列表
numbers = seq_lens

# 计算直方图
# bins 参数定义了区间数或区间边界
hist, bin_edges = np.histogram(numbers, bins=[84,150,160,180,345])

# 计算每个区间的频率
frequency = hist / len(numbers)

# 打印直方图和频率
print("Bin Edges:", bin_edges)
print("Histogram:", hist)
print("Frequency:", frequency)



Bin Edges: [ 84 150 160 180 345]
Histogram: [120 316  93 150]
Frequency: [0.17673049 0.46539028 0.13696613 0.22091311]


 时长统一为150帧、5s。原始帧率为30fps，降低fps增加数据量--6fps 翻成5倍的数据量 和stage0_1obj对齐。